Data Loading and Initial Cleaning

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style('whitegrid')

# Define the path to your data (assuming it's in a 'data' folder outside of your repo)
DATA_PATH = r'C:\Users\Home-User\Downloads\data\raw_analyst_ratings.csv'

# Load the dataset
try:
    df = pd.read_csv(DATA_PATH)
    print(f"Dataset loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns.")
except FileNotFoundError:
    print(f"Error: Dataset not found at {DATA_PATH}. Please check the file path.")
    # If the file is missing, you might load your dummy data here for testing.

# Display initial info
df.info()

Dataset loaded successfully with 1407328 rows and 6 columns.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407328 entries, 0 to 1407327
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Unnamed: 0  1407328 non-null  int64 
 1   headline    1407328 non-null  object
 2   url         1407328 non-null  object
 3   publisher   1407328 non-null  object
 4   date        1407328 non-null  object
 5   stock       1407328 non-null  object
dtypes: int64(1), object(5)
memory usage: 64.4+ MB
